In [1]:
pip install plotly-express

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.10.2+cpu', '4.17.0', '0.11.6')

In [4]:
data_file = 'C:\\Users\\TANIYA JAIN\\arxiv-metadata-oai-snapshot.json//arxiv-metadata-oai-snapshot.json'

""" Using `yield` to load the JSON file in a loop to prevent Python memory issues if JSON is loaded directly"""

def get_metadata():
    with open(data_file, 'r') as f:
        for line in f:
            yield line

In [5]:
metadata = get_metadata()
for paper in metadata:
    for k, v in json.loads(paper).items():
        print(f'{k}: {v} \n')
    break

id: 0704.0001 

submitter: Pavel Nadolsky 

authors: C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan 

title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies 

comments: 37 pages, 15 figures; published version 

journal-ref: Phys.Rev.D76:013009,2007 

doi: 10.1103/PhysRevD.76.013009 

report-no: ANL-HEP-PR-07-12 

categories: hep-ph 

license: None 

abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
m

In [6]:
category_map = {'astro-ph': 'Astrophysics',
                'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
                'astro-ph.EP': 'Earth and Planetary Astrophysics',
                'astro-ph.GA': 'Astrophysics of Galaxies',
                'astro-ph.HE': 'High Energy Astrophysical Phenomena',
                'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
                'astro-ph.SR': 'Solar and Stellar Astrophysics',
                'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
                'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
                'cond-mat.mtrl-sci': 'Materials Science',
                'cond-mat.other': 'Other Condensed Matter',
                'cond-mat.quant-gas': 'Quantum Gases',
                'cond-mat.soft': 'Soft Condensed Matter',
                'cond-mat.stat-mech': 'Statistical Mechanics',
                'cond-mat.str-el': 'Strongly Correlated Electrons',
                'cond-mat.supr-con': 'Superconductivity',
                'cs.AI': 'Artificial Intelligence',
                'cs.AR': 'Hardware Architecture',
                'cs.CC': 'Computational Complexity',
                'cs.CE': 'Computational Engineering, Finance, and Science',
                'cs.CG': 'Computational Geometry',
                'cs.CL': 'Computation and Language',
                'cs.CR': 'Cryptography and Security',
                'cs.CV': 'Computer Vision and Pattern Recognition',
                'cs.CY': 'Computers and Society',
                'cs.DB': 'Databases',
                'cs.DC': 'Distributed, Parallel, and Cluster Computing',
                'cs.DL': 'Digital Libraries',
                'cs.DM': 'Discrete Mathematics',
                'cs.DS': 'Data Structures and Algorithms',
                'cs.ET': 'Emerging Technologies',
                'cs.FL': 'Formal Languages and Automata Theory',
                'cs.GL': 'General Literature',
                'cs.GR': 'Graphics',
                'cs.GT': 'Computer Science and Game Theory',
                'cs.HC': 'Human-Computer Interaction',
                'cs.IR': 'Information Retrieval',
                'cs.IT': 'Information Theory',
                'cs.LG': 'Machine Learning',
                'cs.LO': 'Logic in Computer Science',
                'cs.MA': 'Multiagent Systems',
                'cs.MM': 'Multimedia',
                'cs.MS': 'Mathematical Software',
                'cs.NA': 'Numerical Analysis',
                'cs.NE': 'Neural and Evolutionary Computing',
                'cs.NI': 'Networking and Internet Architecture',
                'cs.OH': 'Other Computer Science',
                'cs.OS': 'Operating Systems',
                'cs.PF': 'Performance',
                'cs.PL': 'Programming Languages',
                'cs.RO': 'Robotics',
                'cs.SC': 'Symbolic Computation',
                'cs.SD': 'Sound',
                'cs.SE': 'Software Engineering',
                'cs.SI': 'Social and Information Networks',
                'cs.SY': 'Systems and Control',
                'econ.EM': 'Econometrics',
                'eess.AS': 'Audio and Speech Processing',
                'eess.IV': 'Image and Video Processing',
                'eess.SP': 'Signal Processing',
                'gr-qc': 'General Relativity and Quantum Cosmology',
                'hep-ex': 'High Energy Physics - Experiment',
                'hep-lat': 'High Energy Physics - Lattice',
                'hep-ph': 'High Energy Physics - Phenomenology',
                'hep-th': 'High Energy Physics - Theory',
                'math.AC': 'Commutative Algebra',
                'math.AG': 'Algebraic Geometry',
                'math.AP': 'Analysis of PDEs',
                'math.AT': 'Algebraic Topology',
                'math.CA': 'Classical Analysis and ODEs',
                'math.CO': 'Combinatorics',
                'math.CT': 'Category Theory',
                'math.CV': 'Complex Variables',
                'math.DG': 'Differential Geometry',
                'math.DS': 'Dynamical Systems',
                'math.FA': 'Functional Analysis',
                'math.GM': 'General Mathematics',
                'math.GN': 'General Topology',
                'math.GR': 'Group Theory',
                'math.GT': 'Geometric Topology',
                'math.HO': 'History and Overview',
                'math.IT': 'Information Theory',
                'math.KT': 'K-Theory and Homology',
                'math.LO': 'Logic',
                'math.MG': 'Metric Geometry',
                'math.MP': 'Mathematical Physics',
                'math.NA': 'Numerical Analysis',
                'math.NT': 'Number Theory',
                'math.OA': 'Operator Algebras',
                'math.OC': 'Optimization and Control',
                'math.PR': 'Probability',
                'math.QA': 'Quantum Algebra',
                'math.RA': 'Rings and Algebras',
                'math.RT': 'Representation Theory',
                'math.SG': 'Symplectic Geometry',
                'math.SP': 'Spectral Theory',
                'math.ST': 'Statistics Theory',
                'math-ph': 'Mathematical Physics',
                'nlin.AO': 'Adaptation and Self-Organizing Systems',
                'nlin.CD': 'Chaotic Dynamics',
                'nlin.CG': 'Cellular Automata and Lattice Gases',
                'nlin.PS': 'Pattern Formation and Solitons',
                'nlin.SI': 'Exactly Solvable and Integrable Systems',
                'nucl-ex': 'Nuclear Experiment',
                'nucl-th': 'Nuclear Theory',
                'physics.acc-ph': 'Accelerator Physics',
                'physics.ao-ph': 'Atmospheric and Oceanic Physics',
                'physics.app-ph': 'Applied Physics',
                'physics.atm-clus': 'Atomic and Molecular Clusters',
                'physics.atom-ph': 'Atomic Physics',
                'physics.bio-ph': 'Biological Physics',
                'physics.chem-ph': 'Chemical Physics',
                'physics.class-ph': 'Classical Physics',
                'physics.comp-ph': 'Computational Physics',
                'physics.data-an': 'Data Analysis, Statistics and Probability',
                'physics.ed-ph': 'Physics Education',
                'physics.flu-dyn': 'Fluid Dynamics',
                'physics.gen-ph': 'General Physics',
                'physics.geo-ph': 'Geophysics',
                'physics.hist-ph': 'History and Philosophy of Physics',
                'physics.ins-det': 'Instrumentation and Detectors',
                'physics.med-ph': 'Medical Physics',
                'physics.optics': 'Optics',
                'physics.plasm-ph': 'Plasma Physics',
                'physics.poph': 'Popular Physics',
                'physics.soc-ph': 'Physics and Society',
                'physics.space-ph': 'Space Physics',
                'q-bio.BM': 'Biomolecules',
                'q-bio.CB': 'Cell Behavior',
                'q-bio.GN': 'Genomics',
                'q-bio.MN': 'Molecular Networks',
                'q-bio.NC': 'Neurons and Cognition',
                'q-bio.OT': 'Other Quantitative Biology',
                'q-bio.PE': 'Populations and Evolution',
                'q-bio.QM': 'Quantitative Methods',
                'q-bio.SC': 'Subcellular Processes',
                'q-bio.TO': 'Tissues and Organs',
                'q-fin.CP': 'Computational Finance',
                'q-fin.EC': 'Economics',
                'q-fin.GN': 'General Finance',
                'q-fin.MF': 'Mathematical Finance',
                'q-fin.PM': 'Portfolio Management',
                'q-fin.PR': 'Pricing of Securities',
                'q-fin.RM': 'Risk Management',
                'q-fin.ST': 'Statistical Finance',
                'q-fin.TR': 'Trading and Market Microstructure',
                'quant-ph': 'Quantum Physics',
                'stat.AP': 'Applications',
                'stat.CO': 'Computation',
                'stat.ME': 'Methodology',
                'stat.ML': 'Machine Learning',
                'stat.OT': 'Other Statistics',
                'stat.TH': 'Statistics Theory'}
                

In [14]:
titles = []
abstracts = []

# Consider all categories in the `category_map` to be used during training and prediction
# paper_categories = np.array(list(category_map.keys())).flatten()

# # Consider specific paper categories to be used during training and prediction
paper_categories = [#"cs.AI", # Artificial Intelligence
                     #"cs.CV", # Computer Vision and Pattern Recognition
"cs.LG" # Machine Learning
                   ]
metadata = get_metadata()
for paper in tqdm(metadata):
    paper_dict = json.loads(paper)
    category = paper_dict.get('categories')
    try:
        year = int(paper_dict.get('journal-ref')[-4:])
        if category in paper_categories and 2010<year<2021:
            titles.append(paper_dict.get('title'))
            abstracts.append(paper_dict.get('abstract').replace("\n",""))
    except:
        pass 

len(titles), len(abstracts)

0it [00:00, ?it/s]

(94, 94)

In [15]:
papers = pd.DataFrame({
    'title': titles,
    'abstract': abstracts,
})

papers = papers[['abstract', 'title']]
papers.columns = ['input_text', 'target_text']
papers = papers.dropna()

del titles, abstracts
papers

,input_text,target_text
0,In this paper we present a new algorithm for...,Geometric Decision Tree
1,Margin theory provides one of the most popul...,On the Doubt about Margin Explanation of Boosting
2,We consider the problem of energy-efficient ...,Fast Reinforcement Learning for Energy-Efficie...
3,To classify time series by nearest neighbors...,Time Series Classification by Class-Specific M...
4,Ordinal regression is commonly formulated as...,Transductive Ordinal Regression
...,...,...
89,Recent breakthroughs in Deep Neural Networks...,"AdaDeep: A Usage-Driven, Automated Deep Model ..."
90,The Symbolic Aggregate approXimation (SAX) i...,Modifying the Symbolic Aggregate Approximation...
91,"In continual learning (CL), a learner is fac...",Continual Learning in Low-rank Orthogonal Subs...
92,The current expansion of theory and research...,Augmenting Organizational Decision-Making with...


In [16]:
pip install simpletransformers

Note: you may need to restart the kernel to use updated packages.


In [17]:
%%time


from simpletransformers.seq2seq import Seq2SeqModel

eval_df = papers.sample(frac=0.1, random_state=42)
train_df = papers.drop(eval_df.index)

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 6,
    "num_train_epochs": 3,
    "fp16": False,

}


# Create a Bart-base model
model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base", use_cuda=False,
                    args=model_args)

Wall time: 12.3 s


In [18]:
%%time
import pandas
from sklearn import model_selection
# Train the model
model.train_model(train_df)

filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# Evaluate the model
result = model.eval_model(eval_df)
print(result)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/85 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/15 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to outputs/.


NameError: name 'pickle' is not defined

In [14]:
for _ in range(250):

    random_idx = random.randint(0, len(eval_df)-1)

    abstract = eval_df.iloc[random_idx]['input_text']
    true_title = eval_df.iloc[random_idx]['target_text']

    # Predict with trained BART model
    predicted_title = model.predict([abstract])[0]

    print(f'True Title: {true_title}\n')
    print(f'Predicted Title: {predicted_title}\n')
    print(f'Abstract: {abstract}\n\n\n')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: Preference-Based Monte Carlo Tree Search

Predicted Title: A Monte Carlo Tree Search: A Unique and Unique Approach

Abstract:   Monte Carlo tree search (MCTS) is a popular choice for solving sequentialanytime problems. However, it depends on a numeric feedback signal, which canbe difficult to define. Real-time MCTS is a variant which may only rarelyencounter states with an explicit, extrinsic reward. To deal with such cases,the experimenter has to supply an additional numeric feedback signal in theform of a heuristic, which intrinsically guides the agent. Recent work hasshown evidence that in different areas the underlying structure is ordinal andnot numerical. Hence erroneous and biased heuristics are inevitable, especiallyin such domains. In this paper, we propose a MCTS variant which only depends onqualitative feedback, and therefore opens up new applications for MCTS. We alsofind indications that translating absolute into ordinal feedback may bebeneficial. Using a puzzl

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: The SP theory of intelligence and the representation and processing of
  knowledge in the brain

Predicted Title: A SP theory of intelligence: An overview and partial model

Abstract:   The "SP theory of intelligence", with its realisation in the "SP computermodel", aims to simplify and integrate observations and concepts acrossAI-related fields, with information compression as a unifying theme. This paperdescribes how abstract structures and processes in the theory may be realisedin terms of neurons, their interconnections, and the transmission of signalsbetween neurons. This part of the SP theory -- "SP-neural" -- is a tentativeand partial model for the representation and processing of knowledge in thebrain. In the SP theory (apart from SP-neural), all kinds of knowledge arerepresented with "patterns", where a pattern is an array of atomic symbols inone or two dimensions. In SP-neural, the concept of a "pattern" is realised asan array of neurons called a "pattern assembly

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

True Title: Autonomous Self-Explanation of Behavior for Interactive Reinforcement
  Learning Agents

Predicted Title: A Instruction-Based Behavior Explanation

Abstract:   In cooperation, the workers must know how co-workers behave. However, anagent's policy, which is embedded in a statistical machine learning model, ishard to understand, and requires much time and knowledge to comprehend.Therefore, it is difficult for people to predict the behavior of machinelearning robots, which makes Human Robot Cooperation challenging. In thispaper, we propose Instruction-based Behavior Explanation (IBE), a method toexplain an autonomous agent's future behavior. In IBE, an agent canautonomously acquire the expressions to explain its own behavior by reusing theinstructions given by a human expert to accelerate the learning of the agent'spolicy. IBE also enables a developmental agent, whose policy may change duringthe cooperation, to explain its own behavior with sufficient time granularity.





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: Autonomous Self-Explanation of Behavior for Interactive Reinforcement
  Learning Agents

Predicted Title: A Instruction-Based Behavior Explanation

Abstract:   In cooperation, the workers must know how co-workers behave. However, anagent's policy, which is embedded in a statistical machine learning model, ishard to understand, and requires much time and knowledge to comprehend.Therefore, it is difficult for people to predict the behavior of machinelearning robots, which makes Human Robot Cooperation challenging. In thispaper, we propose Instruction-based Behavior Explanation (IBE), a method toexplain an autonomous agent's future behavior. In IBE, an agent canautonomously acquire the expressions to explain its own behavior by reusing theinstructions given by a human expert to accelerate the learning of the agent'spolicy. IBE also enables a developmental agent, whose policy may change duringthe cooperation, to explain its own behavior with sufficient time granularity.





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: A Survey of Multi-Objective Sequential Decision-Making

Predicted Title: A Multi-Objective Methods for Planning and Learning

Abstract:   Sequential decision-making problems with multiple objectives arise naturallyin practice and pose unique challenges for research in decision-theoreticplanning and learning, which has largely focused on single-objective settings.This article surveys algorithms designed for sequential decision-makingproblems with multiple objectives. Though there is a growing body of literatureon this subject, little of it makes explicit under what circumstances specialmethods are needed to solve multi-objective problems. Therefore, we identifythree distinct scenarios in which converting such a problem to asingle-objective one is impossible, infeasible, or undesirable. Furthermore, wepropose a taxonomy that classifies multi-objective methods according to theapplicable scenario, the nature of the scalarization function (which projectsmulti-objective values to